In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.callback.tensorboard import *

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import datetime
import pandas as pd
import numpy as np

In [5]:
path = Path('../data/train')

In [6]:
path.ls()

(#3) [Path('../data/train/healthy_wheat'),Path('../data/train/leaf_rust'),Path('../data/train/stem_rust')]

In [7]:
def sort_image_by_dimensions(path, size):
    w, h = Image.open(path).size
    return w >= size and h >= size

In [8]:
image_size = 460

In [9]:
# get_files??

In [10]:
# get_image_files??

In [11]:
cbs = TensorBoardCallback(projector=True)

In [12]:
def correct_sized_images(path):
    return [img for img in get_image_files(path) if sort_image_by_dimensions(img, image_size)]

In [13]:
def get_y(r): return parent_label(r).split(" ")

In [14]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                        splitter=RandomSplitter(seed=42),
                        get_items=correct_sized_images,
                        get_y=get_y,
                        item_tfms=Resize(image_size),
                        batch_tfms=aug_transforms(size=size, min_scale=0.75))

    return dblock.dataloaders(path, bs=bs, num_workers=0)

In [15]:
batch_size = 64
dls = get_dls(batch_size, 64)

In [16]:
# dls.show_batch(max_n=4, nrows=1)

In [17]:
man_in = tensor([[-3.4, 1.8, -3.4],
                 [0.6104, -1.1992, -2.6289]], dtype=torch.double)

man_targ = tensor([[0, 1, 0],
                   [0, 1, 0]], dtype=torch.double)

In [18]:
# x,y = dls.train.one_batch()

In [19]:
cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')

In [20]:
from torch.nn.modules import loss

class smooth_binary_cross_entropy(loss.BCEWithLogitsLoss):

    # def __init__(self,
    #              pos_weight: Optional[Tensor] = None,
    #              reduction: str = 'mean',
    #              thresh=0.5) -> None:
    #     store_attr()
    #
    #     self.pos_weight = pos_weight
    #     self.reduction = reduction
    #     self.thresh = thresh

    pos_weight = torch.ones(3, device=cuda0)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        c = target.shape[1]
        eps = 0.1
        smoothed_target = torch.where(target==1, 1-(eps+(eps/c)), eps/c)
        return F.binary_cross_entropy_with_logits(input,
                                                  smoothed_target,
                                                  pos_weight=self.pos_weight,
                                                  reduction=self.reduction)

    def activation(self, x): return torch.sigmoid(x)


    # device = inputs.device
    #
    # c = targets.shape[1]
    # eps = 0.1
    # threshold = 0.5



    # inputs_sig = inputs.sigmoid()
    
    # print(f"inputs_sig:\n{inputs_sig}\n")



    # smoothed_targets = torch.where(targets==1, 1-(eps+(eps/c)), eps/c)
    #
    # pos_weight = torch.ones(3, device=device)
    #
    # tst = BCEWithLogitsLossFlat(pos_weight=pos_weight)
    # loss = tst(inputs, smoothed_targets)



    # smoothed_targets_two = targets * (1.0 - eps) + 0.5 * eps
    
    # loss = activs/smoothed_targets
    
    # print(f"smoothed_targets:\n{smoothed_targets}\n")
    # print(f"targets:\n{targets}\n")
    # print(f"divided:\n{(inputs_sig/smoothed_targets)}\n")

    # pred = torch.sigmoid(inputs)
    # bce_loss_yang_zhang = -(pred.log()*smoothed_targets + (1-smoothed_targets)*(1-pred).log()).mean()
    # return bce_loss_yang_zhang
    # print(bce_loss_yang_zhang)
    
#     bce_loss = -torch.where(targets==1, inputs_sig, 1-inputs_sig).log().mean()
#     loss = -torch.where(targets==1, (inputs_sig/smoothed_targets), (inputs_sig/smoothed_targets)).log()
#     loss = -(inputs_sig/smoothed_targets).log()
    
#     print(f"bce_loss: {bce_loss}")
#     print(f"bce_loss_smoothed: {bce_loss_smoothed}")
#     print(f"bce_res: {bce_res}")
    
    # return F.binary_cross_entropy_with_logits(inputs, smoothed_targets)
#     return F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#     return loss
#     return loss*(eps/c)+(1-eps)

In [21]:
# smooth_binary_cross_entropy(man_in, man_targ)

In [22]:
# pos_weight = torch.ones(3)
# tst = BCEWithLogitsLossFlat(pos_weight=pos_weight, reduction='none')
# tst(man_in, man_targ)

In [23]:
# F.binary_cross_entropy_with_logits??

In [24]:
from fastai.callback.fp16 import *
learn = cnn_learner(dls,
                    resnet50,
                    metrics=partial(accuracy_multi, thresh=0.2),
                    loss_func=smooth_binary_cross_entropy).to_fp16()

In [25]:
# print(x.shape)
# activs = learn.model(x)
# activs.shape

In [26]:
# activs[0],y[0]

In [27]:
# loss_func = nn.BCEWithLogitsLoss(reduction='none')
# loss = loss_func(activs,y)
# loss

In [28]:
# smooth_binary_cross_entropy(activs[0],y[0])

In [29]:
# preds,_ = learn.get_preds(dl=[(x,y)])
# print(preds[0],preds[0].sum())
# print(y[0])

In [30]:
# print(F.binary_cross_entropy_with_logits(preds, y))

In [31]:
# smooth_binary_cross_entropy(preds, y)

In [32]:
# F.binary_cross_entropy_with_logits??

In [33]:
#learn.lr_find()

In [34]:
# with learn.no_bar():
#     learn.fine_tune(1, freeze_epochs=0)

In [35]:
with learn.no_bar():
    learn.fine_tune(10, freeze_epochs=3)

[0, 1.0581037998199463, 0.8305637836456299, 0.3672839403152466, '01:24']
[1, 0.9656287431716919, 0.7546173930168152, 0.4475308656692505, '01:20']


KeyboardInterrupt: 

In [ ]:
learn.dls = get_dls(batch_size, 128)

In [ ]:
with learn.no_bar():
    learn.fine_tune(10)

In [ ]:
learn.dls = get_dls(batch_size, 256)

In [ ]:
with learn.no_bar():
    learn.fine_tune(40)

In [ ]:
learn.loss_func

In [ ]:
learn.recorder.plot_loss()

In [ ]:
x,y = dls.one_batch()

In [ ]:
import random as rnd

preds,t = learn.get_preds(dl=[(x,y)])
rand_idx = rnd.randint(0, len(preds))-1
print(preds[rand_idx], t[rand_idx])

In [ ]:
preds[0].sum()

In [ ]:
learn.model(x).shape

In [ ]:
model = learn.model
time_now = datetime.datetime.now().strftime(format='%H%M%S')

export_path = Path('../models/exported')

model_filename = f"{export_path}/{time_now}-export.pkl"
learn.export(fname=model_filename)

In [ ]:
learn.dls.vocab